In [20]:
from typing import Callable, Dict, Iterable, Self
from numpy.typing import NDArray
import datasets
import tensorflow as tf
import numpy as np
import einops
import pandas as pd
import pickle
import sklearn.metrics as skm

# import flagon

In [2]:
with open("data/solar_home_data.pkl", 'rb') as f:
    data = pickle.load(f)

def get_customer_data(customer=1):
    idx = np.arange(24, len(data[customer]))
    expanded_idx = np.array([np.arange(i - 24, i - 1) for i in idx])
    return data[customer][expanded_idx].reshape(len(data[customer]) - 24, -1), data[customer][idx, 0]

In [3]:
get_customer_data(1)[0].shape

(8736, 92)

In [31]:
def create_model() -> tf.keras.Model:
    inputs = tf.keras.Input((92))
    # x = tf.keras.layers.Flatten()(inputs)
    x = tf.keras.layers.Reshape((23, 4))(inputs)
    x = tf.keras.layers.Conv1D(32, 5, activation="relu")(x)
    x = tf.keras.layers.Conv1D(64, 3, activation="relu")(x)
    x = tf.keras.layers.Conv1D(64, 2, activation="relu")(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(50, activation="relu")(x)
    x = tf.keras.layers.Dense(1, activation="linear")(x)
    model = tf.keras.Model(inputs=inputs, outputs=x)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
        loss="mean_squared_error",
        metrics=["mean_absolute_error", tf.keras.metrics.RootMeanSquaredError()]
    )
    return model

In [5]:
X, Y = get_customer_data(1)

In [32]:
model = create_model()
model.fit(X, Y, epochs=5)

Epoch 1/5
273/273 [==============================] - 1s 1ms/step - loss: 73.3537 - mean_absolute_error: 1.6102 - root_mean_squared_error: 8.5647
Epoch 2/5
273/273 [==============================] - 0s 1ms/step - loss: 1.2202 - mean_absolute_error: 0.8361 - root_mean_squared_error: 1.1046
Epoch 3/5
273/273 [==============================] - 0s 1ms/step - loss: 1.1158 - mean_absolute_error: 0.8018 - root_mean_squared_error: 1.0563
Epoch 4/5
273/273 [==============================] - 0s 1ms/step - loss: 1.0184 - mean_absolute_error: 0.7628 - root_mean_squared_error: 1.0092
Epoch 5/5
273/273 [==============================] - 0s 1ms/step - loss: 1.8265 - mean_absolute_error: 1.0221 - root_mean_squared_error: 1.3515


In [94]:
Y[Y < 0]

array([-0.594, -0.862, -1.591, ..., -0.444, -0.283, -0.364])